# Decision Tree / Random Forest

Binary classification of the risk involved with each loan handed out. Features of the loan includes the month, loan amount, term length, and product type. Features of the client are age, gender, education, presence of children, region, income, family status, phone operator, and whether the loanee is an existing client. For this dataset, the Decision Tree and Random Forest algorithms will be used to predict whether a client is "good" or "bad" based on the features above.

### Import Libraries

In [1]:
import pandas as pd
import seaborn as sns

### Import Dataset(s)

In [2]:
df = pd.read_csv('../data/client-risk.csv')

In [3]:
df.head()

,month,credit_amount,credit_term,age,sex,education,product_type,having_children_flg,region,income,family_status,phone_operator,is_client,bad_client_target
0,1,7000,12,39,male,Secondary special education,Cell phones,0,2,21000,Another,0,0,0
1,1,19000,6,20,male,Secondary special education,Household appliances,1,2,17000,Another,3,1,0
2,1,29000,12,23,female,Secondary special education,Household appliances,0,2,31000,Another,2,0,0
3,1,10000,12,30,male,Secondary special education,Cell phones,1,2,31000,Unmarried,3,1,0
4,1,14500,12,25,female,Higher education,Cell phones,0,2,26000,Married,0,1,0
